In [1]:
!pip install flask-ngrok

In [2]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 3.4 MB/s 


In [ ]:
import pandas as pd
import pickle
import json
from flask import Flask, request
from flask_ngrok import run_with_ngrok
from category_encoders       import OneHotEncoder

In [ ]:
class Fraud: 
    def __init__(self):
        self.encoder = pickle.load(open('/content/drive/My Drive/Projetos_ML/fraud/deploy/enc_fraud.pkl', 'rb'))
              
    def data_preparation(self, data_set):
        # selected columns
        final_columns = ['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
        data_set2 = data_set[final_columns]

        # OneHotEncoder
        data_set3 = self.encoder.transform(data_set2)
        for col in ['type_CASH_IN','type_PAYMENT','type_CASH_OUT','type_TRANSFER','type_DEBIT']:
            data_set3[col] = data_set3[col].astype('category')       
              
        return data_set3
    
    def get_prediction(self, model, original_data, test_data):
        pred = model.predict(test_data)
        original_data['prediction'] = pred
        
        return original_data.to_json(orient="records", date_format="iso")

In [ ]:
# load model
model = pickle.load(open('/content/drive/My Drive/Projetos_ML/fraud/deploy/model_rf_fraud.pkl', 'rb'))
# instanciate flask
app = Flask( __name__ )
run_with_ngrok(app)

@app.route('/predict', methods=['POST'])  
def predict():
    ori_json = request.get_json()

    # collect data
    if ori_json:
      #aux_json = json.loads(ori_json)
      df_raw = pd.DataFrame.from_dict(ori_json, orient="columns")
      # Instantiate Fraud class
      pipeline = Fraud()

      # data prepatarion
      df = pipeline.data_preparation(df_raw)

      # prediction
      pred = pipeline.get_prediction(model, df_raw, df)
    
      return pred
    else:
      return Reponse('{}', status=200, mimetype='application/json')

if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d26e-35-192-157-201.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [31/Aug/2021 03:22:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2021 03:25:29] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2021 03:39:39] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2021 03:56:20] "POST /predict HTTP/1.1" 200 -
